In [1]:
from app import create_app, db
from app.model import Airports, Flights

C:\Users\tv\.conda\envs\geo_env\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [3]:
app = create_app()

C:\Users\tv\.conda\envs\geo_env\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [5]:
with app.app_context():
    iata = "ATL"
    airport = Airports.query.filter_by(iata=iata).first()

In [7]:
airport.name

'Atlanta Hartsfield-Jackson International Airport'

In [11]:
from sqlalchemy import func, distinct 

with app.app_context():
    res = Flights.query.with_entities(Flights.origin_airport_iata, Flights.destination_airport_iata,
                                     func.count(distinct(Flights.number)))\
    .filter(Flights.destination_airport_iata.isnot(None))\
    .filter(Flights.origin_airport_iata.isnot(None))\
    .group_by(Flights.origin_airport_iata, Flights.destination_airport_iata)\
    .having(func.count(distinct(Flights.number)) > 0).all()

In [40]:
len(res)

9656

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(res)
df.columns = ['origin', 'destination', 'count']
df

,origin,destination,count
0,AAF,BHM,1
1,ABE,AFW,1
2,ABE,ATL,1
3,ABE,CLT,1
4,ABE,IAD,1
...,...,...,...
9651,ZRH,YUL,1
9652,ZSA,NAS,1
9653,ZSJ,YQT,1
9654,ZSJ,YXL,1


In [15]:
import numpy as np
df[['iata_1', 'iata_2']] = np.sort(df[['origin', 'destination']], axis=1)
df

,origin,destination,count,iata_1,iata_2
0,AAF,BHM,1,AAF,BHM
1,ABE,AFW,1,ABE,AFW
2,ABE,ATL,1,ABE,ATL
3,ABE,CLT,1,ABE,CLT
4,ABE,IAD,1,ABE,IAD
...,...,...,...,...,...
9651,ZRH,YUL,1,YUL,ZRH
9652,ZSA,NAS,1,NAS,ZSA
9653,ZSJ,YQT,1,YQT,ZSJ
9654,ZSJ,YXL,1,YXL,ZSJ


In [16]:
df.sort_values(['iata_1', 'iata_2'])

,origin,destination,count,iata_1,iata_2
39,ABY,AAF,1,AAF,ABY
0,AAF,BHM,1,AAF,BHM
1,ABE,AFW,1,ABE,AFW
91,AFW,ABE,1,ABE,AFW
2,ABE,ATL,1,ABE,ATL
...,...,...,...,...,...
9610,YYZ,YYT,1,YYT,YYZ
9534,YYY,YZV,1,YYY,YZV
9621,YZV,YYY,1,YYY,YZV
9636,ZIH,YYZ,1,YYZ,ZIH


In [17]:
df = df.groupby(['iata_1', 'iata_2'], as_index=False)['count'].sum()

In [18]:
df

,iata_1,iata_2,count
0,AAF,ABY,1
1,AAF,BHM,1
2,ABE,AFW,2
3,ABE,ATL,2
4,ABE,BRU,1
...,...,...,...
6053,YYR,YYT,4
6054,YYT,YYZ,1
6055,YYY,YZV,2
6056,YYZ,ZIH,1


In [20]:
with app.app_context():
    airports = Airports.query.all()

In [21]:
len(airports)

4600

In [22]:
airport_dict = {x.iata: x.wkb_geometry for x in airports}

In [23]:
df['iata_1_coord'] = df['iata_1'].map(airport_dict)
df['iata_2_coord'] = df['iata_2'].map(airport_dict)

In [28]:
df.shape

(6058, 5)

In [29]:
df = df[(~df['iata_1_coord'].isnull()) & (~df['iata_2_coord'].isnull())]

In [30]:
df.shape

(5954, 5)

In [31]:
from geoalchemy2.shape import to_shape
from shapely.geometry import LineString

In [32]:
def create_linestring(row):
    return LineString([to_shape(row.iata_1_coord), to_shape(row.iata_2_coord)])

In [33]:
df['geometry'] = df.apply(create_linestring, axis=1)

C:\Users\tv\.conda\envs\geo_env\lib\site-packages\pandas\core\dtypes\cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\tv\AppData\Local\Temp/ipykernel_19408/1403314551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(create_linestring, axis=1)


In [35]:
import geopandas as gpd

gdf = gpd.GeoDataFrame(df, geometry="geometry", crs=4326)

In [36]:
gdf = gdf.drop(['iata_1_coord', 'iata_2_coord'], axis=1)

In [37]:
gdf

,iata_1,iata_2,count,geometry
0,AAF,ABY,1,"LINESTRING (-84.98333 29.73333, -84.19440 31.5..."
1,AAF,BHM,1,"LINESTRING (-84.98333 29.73333, -86.75230 33.5..."
2,ABE,AFW,2,"LINESTRING (-75.44040 40.65230, -97.31000 32.9..."
3,ABE,ATL,2,"LINESTRING (-75.44040 40.65230, -84.42800 33.6..."
4,ABE,BRU,1,"LINESTRING (-75.44040 40.65230, 4.48444 50.90138)"
...,...,...,...,...
6052,YYJ,YYZ,2,"LINESTRING (-123.42500 48.64694, -79.62701 43...."
6053,YYR,YYT,4,"LINESTRING (-60.42580 53.31916, -52.75190 47.6..."
6054,YYT,YYZ,1,"LINESTRING (-52.75190 47.61861, -79.62701 43.6..."
6055,YYY,YZV,2,"LINESTRING (-68.20800 48.60861, -66.26550 50.2..."


In [38]:
gdf[gdf['count'] > 10].explore(column="count")